# Задание 1

Вам даны предобработанные и уже нормализованные документы (один документ - одно предложение). Вычислите значения PPMI для заданных пар термов на основе простых частот.

### Считываем список документов

In [16]:
text = '''brewhas california crush game late originator paul runs runs scored second six straight ton
article dietz dietz fred j mccall mccall paul writes
highley larry let lhighley paul utility wrote
brian bring bskendig hudson hudson jr kendig paul specifically writes writes
argument bangkok come directly going hudson hudson jr lines moral paul writes
article crowley defend digitized lines paul pdc perhaps types would writes
angry case conditt could get instead like paul see tell things upset writes
article champion distribution havemann minnesota organization paul paul twins usa world writes
article article lines olson paul paul writes
article article harvey mlee paul pharvey writes
article hudson hudson jr paul writes writes
arras article de hager hagerp jim jmd paul writes writes
anyone article chips good know paul simundza supplier writes
cyclical hudson hudson jr paul prophecy somewhat suggestion tends writes
aiken article computation gives harvard keywords lab organization paul pimentel university writes
application article considering distribution free large making papresco paul prescod reasonably writes
allan allanh article heim papresco paul prescod writes
article article dick king king paul proberts roberts writes
article chemistry chlorine obvious paul sigurdsson steinly steinn writes
armenians article attacking curious greeks halsall halsall paul serdar spend time writes'''

docs = text.split('\n')

### Задаем функцию вычисления PPMI

In [17]:
from math import log2

N = len(docs)

def PPMI(w0, w1):
  w0_count = 0
  w1_count = 0
  both_count = 0
  for doc in docs:
    words = [word for word in doc.split()]
    if w0 in words:
      w0_count += 1
    if w1 in words:
      w1_count += 1
    if w0 in words and w1 in words:
      both_count += 1
  return round(max(log2((N * both_count) / (w0_count * w1_count)), 0), 3)

### Вычисляем PPMI


Вычислим значение PPMI для термов:
- `hudson` и `jr`
-  `article` и `hudson`
- `paul` и `hudson`

In [18]:
print('PPMI hudson и jr:', PPMI('hudson', 'jr'))
print('PPMI article и hudson:', PPMI('article', 'hudson'))
print('PPMI paul и hudson:', PPMI('paul', 'hudson'))

PPMI hudson и jr: 2.322
PPMI article и hudson: 0
PPMI paul и hudson: 0.0


# Задание 2

Используйте датасет, содержащий id твита, ключевое слово, локацию, непосредственно текст твита и метку класса (явлется ли твит сообщением о катастрофе).

### Считываем данные

Оставляем только интересующие нас колонки - `text` и `target`

In [19]:
import pandas as pd

df = pd.read_csv('tweets.csv')
df = df[['text', 'target']]
df.head()

,text,target
0,"Communal violence in Bhainsa, Telangana. ""Ston...",1
1,Telangana: Section 144 has been imposed in Bha...,1
2,Arsonist sets cars ablaze at dealership https:...,1
3,Arsonist sets cars ablaze at dealership https:...,1
4,"""Lord Jesus, your love brings freedom and pard...",0


### Предобработка текста

Выполните предварительную обработку текста, а именно реализуйте функцию для предобработки сообщений, которая делает с текстом следующее (по аналогии с заданием к теме Классификация):

- приводит текст к нижнему регистру `(.lower)`;
- удаляет стоп-слова `(nltk.corpus.stopwords, "english")`;
- удаляет знаки препинания (регулярное выражение `[^\w\s]`);
- нормализует текст при помощи стеммера Snowball `(nltk.stem.SnowballStemmer('english'))`.


In [20]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [21]:
from nltk import stem
from nltk.corpus import stopwords
import re

stemmer = stem.SnowballStemmer('english')
stopwords = set(stopwords.words('english'))

def preprocess(text):
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()
    words = text.split()
    words =[stemmer.stem(word) for word in words if word not in stopwords]
    text = ' '.join(words)
    return text

Применяем получившуюся функцию к текстам:

In [22]:
df['text'] = df['text'].apply(preprocess)

### Разделение данных на обучающую и тестовую выборки

Разделите датасет на тренировочный и тестовый набор данных с параметрами `test_size = 0.3`, `random_state = 1`

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['target'], test_size=0.3, random_state=1)

### Обучение классификатора

Для векторизации используйте объект `TfidfVectorizer()` (параметры по умолчанию).

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

Обучите классификатор SVM `(LinearSVC)` при `C = 1.4` и `random_state = 1` на обучающей выборке и произведите оценку полученной модели на тестовой.

In [25]:
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

model = LinearSVC(random_state = 1, C = 1.4)
model.fit(X_train, y_train)
predictions = model.predict(X_test)

In [26]:
print(classification_report(y_test, predictions, digits=3))

              precision    recall  f1-score   support

           0      0.917     0.956     0.936      2788
           1      0.757     0.612     0.677       623

    accuracy                          0.893      3411
   macro avg      0.837     0.784     0.806      3411
weighted avg      0.888     0.893     0.889      3411

